In [1]:
# set correct path for imports for this notebook
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# external imports 
import logging
logging.basicConfig(level=logging.INFO)

# local imports
from src.fetch.google_api import fetch_google_places_data
from src.fetch.google_api import fetch_place_id
from src.nlp.extractor_openai import extract_dishes_openai, _cached_extract_single
from src.ranking.scoring import assign_dish_scores
from src.recommendation.recs import form_recommendations

# Clear dish extractor cache to ensure fresh model load
_cached_extract_single.cache_clear()

# Restaurants to test with
r1_id = fetch_place_id("funkyfish berlin")
r2_id = fetch_place_id("standard pizza west berlin")
r3_id = fetch_place_id("borschert berlin")
r4_id = fetch_place_id("Gio's Berlin") 
r5_id = fetch_place_id("hasir kreuzberg")
r6_id = fetch_place_id("paolo pinkel berlin")

# Getting restaurant info and reviews
r1, funky = fetch_google_places_data(r1_id)
r2, pizza = fetch_google_places_data(r2_id)
r3, borch = fetch_google_places_data(r3_id)
r4, gio  = fetch_google_places_data(r4_id)
r5, hasir = fetch_google_places_data(r5_id)
r6, paolo = fetch_google_places_data(r6_id)

INFO:src.fetch.google_api:
Found restaurant: FunkyFisch
Kantstraße 135-136, 10625 Berlin, Deutschland
ID = ChIJU7-KvONQqEcRCmPCCvo-Fg4

INFO:src.fetch.google_api:
Found restaurant: Standard Serious Pizza Charlottenburg
Schlüterstraße 63, 10625 Berlin, Deutschland
ID = ChIJc2uwDI1RqEcRfAsy3YxyfIY

INFO:src.fetch.google_api:
Found restaurant: Borchardt
Französische Str. 47, 10117 Berlin, Deutschland
ID = ChIJVfEmINtRqEcRsch0pslaanQ

INFO:src.fetch.google_api:
Found restaurant: Gio's
Dresdener Str. 16, 10999 Berlin, Deutschland
ID = ChIJ65Liv5JPqEcRodr8aFFNAvE

INFO:src.fetch.google_api:
Found restaurant: Hasir Kreuzberg
Adalbertstraße 10, 10999 Berlin, Deutschland
ID = ChIJZ-8TQ8hPqEcRf0R-vl8UwwY

INFO:src.fetch.google_api:
Found restaurant: Paolo Pinkel
Karl-Marx-Straße 55, 12043 Berlin, Deutschland
ID = ChIJpRECXmtPqEcRJMfokCTNvD8

INFO:src.fetch.google_api:
Found restaurant: FunkyFisch
Kantstraße 135-136, 10625 Berlin, Germany

INFO:src.fetch.google_api:Retrieved 5 reviews for FunkyFi

In [2]:
# Extracting dishes
restaurant = paolo #change restaurant here

%autoawait
review_with_dishes = await extract_dishes_openai(restaurant, True)

INFO:src.nlp.extractor_openai:🚀 Starting async extraction for 5 chunks...


IPython autoawait is `on`, and set to use `asyncio`


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:src.nlp.extractor_openai:🍽️ Extracted from Review #1: none
INFO:src.nlp.extractor_openai:🍽️ Extracted from Review #2: schnitzel, roastbeef, ceviche
INFO:src.nlp.extractor_openai:🍽️ Extracted from Review #3: fries, fried chicken
INFO:src.nlp.extractor_openai:🍽️ Extracted from Review #4: none
INFO:src.nlp.extractor_openai:🍽️ Extracted from Review #5: kimcheese fries, korean fried chicken
INFO:src.nlp.extractor_openai:✅ Completed dish extraction for 5 reviews in 4.17s


In [3]:
assign_dish_scores(paolo)

INFO:src.ranking.scoring:================ SCORING ===============
INFO:src.ranking.scoring:========================================
INFO:src.ranking.scoring:SCORE: Review #2 -- 11p -- schnitzel -- (0 +10 +1)
INFO:src.ranking.scoring:SCORE: Review #2 -- 11p -- roastbeef -- (0 +10 +1)
INFO:src.ranking.scoring:SCORE: Review #2 -- 11p -- ceviche -- (0 +10 +1)
INFO:src.ranking.scoring:========================================
INFO:src.ranking.scoring:SCORE: Review #3 -- 1p -- fries -- (0 +0 +1)
INFO:src.ranking.scoring:SCORE: Review #3 -- 8p -- fried chicken -- (0 +0 +8)
INFO:src.ranking.scoring:========================================
INFO:src.ranking.scoring:========================================
INFO:src.ranking.scoring:SCORE: Review #5 -- 18p -- kimcheese fries -- (0 +10 +8)
INFO:src.ranking.scoring:SCORE: Review #5 -- 37p -- korean fried chicken -- (0 +10 +27)
INFO:src.ranking.scoring:========================================


In [13]:
reviews = paolo

import logging
from typing import Any, Dict, List, Optional

logger = logging.getLogger(__name__)

recommendations: List[Dict[str, Any]] = []

for review in reviews:
    source = review.get("source", "google")
    author = review.get("author")
    url = review.get("url")
    timestamp = review.get("timestamp")

    dishes = review.get("dishes", [])
    if not dishes:
        continue

    for dish in dishes:
        recommendations.append({
            "dish_name": dish.get("name"),
            "ranking": dish.get("ranking"),
            "author": author,
            "source": source,
            "timestamp": timestamp,
            "review_link": url,
        })

# Sort: dishes with None rankings go last
recommendations.sort(
    key=lambda rec: (rec["ranking"] is None, -(rec["ranking"] or 0))
)

logger.info(f"🍽️ Generated {len(recommendations)} recommendations from {len(reviews)} reviews.")
logger.info(f"🍽️ Sorted dish scores are {[rec.get('ranking','') for rec in recommendations]}.")

recommendations


INFO:__main__:🍽️ Generated 7 recommendations from 5 reviews.
INFO:__main__:🍽️ Sorted dish scores are [37, 18, 11, 11, 11, 8, 1].


[{'dish_name': 'korean fried chicken',
  'ranking': 37,
  'author': 'Nadine Co',
  'source': 'Google Reviews',
  'timestamp': 1740520451,
  'review_link': HttpUrl('https://www.google.com/maps/reviews/data=!4m6!14m5!1m4!2m3!1sChdDSUhNMG9nS0VJQ0FnTURnbWZYMHlRRRAB!2m1!1s0x47a84f6b5e0211a5:0x3fbccd2490e8c724')},
 {'dish_name': 'kimcheese fries',
  'ranking': 18,
  'author': 'Nadine Co',
  'source': 'Google Reviews',
  'timestamp': 1740520451,
  'review_link': HttpUrl('https://www.google.com/maps/reviews/data=!4m6!14m5!1m4!2m3!1sChdDSUhNMG9nS0VJQ0FnTURnbWZYMHlRRRAB!2m1!1s0x47a84f6b5e0211a5:0x3fbccd2490e8c724')},
 {'dish_name': 'schnitzel',
  'ranking': 11,
  'author': 'Deer Ozzie',
  'source': 'Google Reviews',
  'timestamp': 1749415038,
  'review_link': HttpUrl('https://www.google.com/maps/reviews/data=!4m6!14m5!1m4!2m3!1sChZDSUhNMG9nS0VNNjQ4T2ltZzlxTUl3EAE!2m1!1s0x47a84f6b5e0211a5:0x3fbccd2490e8c724')},
 {'dish_name': 'roastbeef',
  'ranking': 11,
  'author': 'Deer Ozzie',
  'source': 'Go